In [ ]:
#pip install pandas matplotlib seaborn scikit-learn numpy xgboost scipy rich shap

: 

In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import numpy as np
import pickle
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from scipy.sparse import csr_matrix
from rich.progress import track
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib as mpl
from enum import Enum
import shap
import argparse
from sklearn.model_selection import cross_val_score

/home/ubuntu/.local/lib/python3.9/site-packages/xgboost/core.py:265: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc 2.28+) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(
/home/ubuntu/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
print(os.getcwd())
# Save the model. Create the model folder if it does not exist
os.makedirs('./models_dro', exist_ok=True)


/data/dorothy/amr_ML/models_DRO/models_hypatia_beforeDeletion


In [4]:
# Save the results of the best model to a SVG file
def plot_metrics(y_test, y_pred, classes = ['S', 'R'], filename='metrics.svg', cmap='crest'):
    gs = plt.GridSpec(1, 2)

    cm = confusion_matrix(y_test, y_pred)
    # Change labels from 0, 1 to S and R
    cm_df = pd.DataFrame(cm, index=classes, columns=classes)

    fig = plt.figure(figsize=(12, 4))

    ax0 = fig.add_subplot(gs[0, 0])
    _ = ax0.annotate('A', xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16, fontweight='bold')
    _ = sns.heatmap(cm_df, annot=True, fmt='g', cmap=cmap, linewidths=0.5, annot_kws={"size": 14})  # Increase annotation size here
    _ = plt.title('Confusion Matrix', fontsize=14)  # Title font size
    _ = plt.xlabel('Predicted labels', fontsize=12)  # X-axis label font size
    _ = plt.ylabel('True labels', fontsize=12)  # Y-axis label font size
    _ = plt.xticks(fontsize=10)  # X-tick labels font size
    _ = plt.yticks(fontsize=10)  # Y-tick labels font size

    ax1 = fig.add_subplot(gs[0, 1])
    _ = ax1.annotate('B', xy=(-0.05, 1.02), xycoords="axes fraction", fontsize=16, fontweight='bold')

    clf_report = classification_report(y_test, y_pred, output_dict=True)
    keys_to_plot = [key for key in clf_report.keys() if key not in ('accuracy', 'macro avg', 'weighted avg')]
    df = pd.DataFrame(clf_report, columns=keys_to_plot).T

    rows, cols = df.shape
    mask = np.zeros(df.shape)
    mask[:,cols-1] = True

    ax1 = sns.heatmap(df, mask=mask, annot=True, cmap=cmap, fmt='.4g',
                      vmin=0.0, vmax=1.0, linewidths=2, linecolor='white',
                      annot_kws={"size": 12},  # Increase annotation size here
                      yticklabels=classes)

    # Then, let's add the support column by normalizing the colors in this column
    mask = np.zeros(df.shape)
    mask[:,:cols-1] = True    

    ax1 = sns.heatmap(df, mask=mask, annot=True, cmap=cmap, cbar=False,
                      linewidths=2, linecolor='white', fmt='.0f',
                      vmin=df['support'].min(), vmax=df['support'].sum(),         
                      norm=mpl.colors.Normalize(vmin=df['support'].min(),
                                                vmax=df['support'].sum()),
                      annot_kws={"size": 12},  # Increase annotation size here
                      yticklabels=classes)
            
    _ = plt.title("Classification Report", fontsize=14)  # Title font size
    _ = plt.xticks(rotation=0, fontsize=10)  # X-tick labels font size
    _ = plt.yticks(rotation=360, fontsize=10)  # Y-tick labels font size

    # Save as SVG
    plt.savefig(filename, dpi=300)



In [5]:
# Important Variables
njobs = 21
random_state = 42

In [6]:
import json
with open('/data/dorothy/amr_ML/models_DRO/data/assemblies_test.txt', 'r') as f:
    assemblies = json.load(f)


In [7]:
def load_data(kmer_size = 3):
    print(f"Loading data for kmer size {kmer_size}")

    # print to output file
    with open(output, 'a') as f:
        print(f"Loading data for kmer size {kmer_size}", file=f)
    
    # Make the final concatenated dataset 
    X_dna = np.load(f"/data/dorothy/amr_ML/models_DRO/data/data/kmer_{kmer_size}_flatteneddna_.npy", allow_pickle=True)
    X_protein = np.load(f"/data/dorothy/amr_ML/models_DRO/data/data/kmer_{kmer_size}_flattenedprotein_.npy", allow_pickle=True)
    X_rrna = np.load(f"/data/dorothy/amr_ML/models_DRO/data/data/kmer_{kmer_size}_flattenedrrna_.npy", allow_pickle=True)

    X_rrna_subset = X_rrna[indices]
    X_dna_subset = X_dna[indices]
    X_protein_subset = X_protein[indices]
    
    del X_dna, X_protein, X_rrna

    X = np.concatenate((X_dna_subset, X_rrna_subset, X_protein_subset), axis=1)
    X = X.astype(np.int16)
    y = antibiotic_binary

     # Combine y and group labels for stratification
    stratification_labels = [f"{y_class}_{group}" for y_class, group in zip(y, antibiogram_unique['genus'])]
    
    # Perform the 80-20 split with both stratification and group constraints
    train_indices, test_indices = train_test_split(
        range(len(X)), 
        test_size=0.2, 
        random_state=42, 
        stratify=stratification_labels
    )
    
    # Use these indices to filter the original DataFrame
    #train_df = antibiogram_unique.iloc[train_indices]
    #test_df = antibiogram_unique.iloc[test_indices]
    
    # Optionally, you can retrieve specific columns if needed
    X_train, X_test = X[train_indices], X[test_indices]
    y_train, y_test = y[train_indices], y[test_indices]
    
    X_train = csr_matrix(X_train)
    X_test = csr_matrix(X_test)

    del X, y
    del X_dna_subset, X_protein_subset, X_rrna_subset

    return X_train, X_test, y_train, y_test, kmer_size, train_indices, test_indices

In [16]:
###Aztreonam

In [7]:
antibiogram_file="/data/dorothy/amr_ML/models_DRO/models_hypatia_beforeDeletion/aztreonam_QCed_metadata.csv"
output="./kmer_rf_xgboost_output.txt"
output_final="./models_dro/kmer_rf_xgboost_output_final.txt"

# -------------------------
# Load the data
# -------------------------
antibiogram = pd.read_csv(antibiogram_file)
antibiogram['phenotype'].value_counts()

print(f"Preparing data for antibiotic: {antibiogram['antibiotic'].iloc[0]}")
print("-" * 40)

with open(output, 'w') as f:
    print(f"Preparing data for antibiotic: {antibiogram['antibiotic'].iloc[0]}", file=f)
    print("-" * 40, file=f)

# Remove antibiograms N phenotype
antibiogram = antibiogram[antibiogram['phenotype'] != 'N']
antibiogram = antibiogram[antibiogram['phenotype'] != 'I']
antibiogram = antibiogram[antibiogram['phenotype'] != 'NS']

antibiogram_unique = antibiogram.drop_duplicates(subset='id')

# Convert 'Element' to binary values
antibiotic_binary = antibiogram_unique['phenotype'].map({'S': 0, 'R': 1})
antibiotic_binary = antibiotic_binary.to_numpy().astype(np.int8)


# Find the indices of the elements in the sublist
indices = [assemblies.index(element) for element in antibiogram_unique['id']]

#New
# Resetting index for antibiogram_gentamycin_unique
antibiogram_unique.reset_index(drop=True, inplace=True)
antibiogram_unique['organism_name'] = antibiogram_unique['organism_name'].replace('E.coli and', 'Escherichia coli')
indices_antibiograms=antibiogram_unique.index.tolist()

#New
# Extract the genus (first word) from the organism_name column
antibiogram_unique['genus'] = antibiogram_unique['organism_name'].str.split().str[0]

# Find how many from each genus are there
genera_counts = antibiogram_unique['genus'].value_counts()

# Keep the indexes of each found genus
genera_indices = antibiogram_unique.groupby('genus').apply(lambda x: x.index.tolist())

# Report total number of rows
total_rows = antibiogram_unique.shape[0]


# Get the antibiotic from the first row
antibiotic_name = antibiogram_unique.loc[0, 'antibiotic']  # Assuming 'antibiotic' is the column name

# List of genera to include in the output
genera_list = ['Escherichia', 'Klebsiella', 'Acinetobacter', 'Staphylococcus', 'Pseudomonas', 'Enterobacter', 'Enterococcus']

# Example genera_counts data (replace with actual values from your dataset)
genera_counts = antibiogram_unique['genus'].value_counts()  # Replace df with your actual DataFrame

# Dictionary to store the results
summary_data = {
    'Total number': [total_rows],  # Total number of rows in your dataset
    'Antibiotic': [antibiotic_name]  # Add the antibiotic name as a row
}

# Populate the dictionary with genus counts, defaulting to 0 if the genus is not present
for genus in genera_list:
    summary_data[genus] = [genera_counts.get(genus, 0)]
    genus_res = antibiogram_unique.loc[antibiogram_unique["genus"]==genus,"phenotype"].value_counts()
    column_wanted=f"{genus}_NumberOfResistantStrains"
    summary_data[column_wanted] = [genus_res.get("R", 0)]
    
# Convert the dictionary to a DataFrame
summary_df = pd.DataFrame(summary_data)

best_kmer_size = 5

X_train, X_test, y_train, y_test, kmer_size, train_indices, test_indices = load_data(best_kmer_size)






Preparing data for antibiotic: aztreonam
----------------------------------------


/tmp/ipykernel_3170338/3331695195.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  antibiogram_unique['organism_name'] = antibiogram_unique['organism_name'].replace('E.coli and', 'Escherichia coli')
/tmp/ipykernel_3170338/3331695195.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  antibiogram_unique['genus'] = antibiogram_unique['organism_name'].str.split().str[0]
/tmp/ipykernel_3170338/3331695195.py:47: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is de

Loading data for kmer size 5


In [8]:
import numpy as np
import xgboost as xgb
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

# Split off validation set from training
X_train2, X_val, y_train2, y_val = train_test_split(
    X_train, y_train, test_size=0.2, stratify=y_train, random_state=42
)

In [9]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from itertools import product

# Define grid
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [4, 6, 8],
    'alpha': [0.2, 0.3],
    'smoothing': [0.9, 0.95]
}

best_score = None
best_model = None
best_params = {}
best_thresh = None

# Grid search over all combinations
for n_est, depth, alpha, smooth in product(param_grid['n_estimators'], param_grid['max_depth'], param_grid['alpha'], param_grid['smoothing']):
    
    # Step 1: Uncertainty weighting
    base_model = xgb.XGBClassifier(n_estimators=100, max_depth=13, random_state=42)
    base_model.fit(X_train2, y_train2)
    y_probs = base_model.predict_proba(X_train2)[:, 1]
    uncertainty = np.abs(0.5 - y_probs)

    cutoff = int(len(y_train2) * alpha)
    worst_indices = np.argsort(uncertainty)[:cutoff]

    sample_weights = np.ones_like(y_train2)
    sample_weights[worst_indices] = 5
    sample_weights[y_train2 == 0] *= 2

    y_smooth = y_train2 * smooth + (1 - smooth) / 2

    # Step 2: DRO model
    model = xgb.XGBRegressor(n_estimators=n_est, max_depth=depth, objective='reg:logistic', random_state=42)
    model.fit(X_train2, y_smooth, sample_weight=sample_weights)

    # Step 3: Threshold sweep with custom F1–FN tradeoff
    lambda_fn = 0.01  # penalty for each FN
    y_probs_test = model.predict(X_test)

    for thresh in np.arange(0.1, 0.7, 0.01):
        y_pred = (y_probs_test > thresh).astype(int)
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        f1 = f1_score(y_test, y_pred)

        # Custom score: prioritize F1, penalize FN
        score = f1 - lambda_fn * fn

        if best_score is None or score > best_score:
            best_score = score
            best_model = model
            best_thresh = thresh
            best_params = {
                'n_estimators': n_est,
                'max_depth': depth,
                'alpha': alpha,
                'smoothing': smooth,
                'threshold': thresh,
                'FP': fp,
                'FN': fn,
                'F1': f1
            }

# Final report
if best_model:
    print(f"\n✅ Best config:")
    for k, v in best_params.items():
        print(f"{k}: {v}")
    y_pred_final = (best_model.predict(X_test) > best_thresh).astype(int)
    print("\nFinal Performance:")
    print(classification_report(y_test, y_pred_final, target_names=["S", "R"]))
else:
    print("❌ No model found (this shouldn't happen).")


✅ Best config:
n_estimators: 200
max_depth: 6
alpha: 0.2
smoothing: 0.9
threshold: 0.1
FP: 91
FN: 1
F1: 0.9248366013071896

Final Performance:
              precision    recall  f1-score   support

           S       1.00      0.81      0.90       490
           R       0.86      1.00      0.92       567

    accuracy                           0.91      1057
   macro avg       0.93      0.91      0.91      1057
weighted avg       0.92      0.91      0.91      1057



In [10]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

print("\nThreshold Sweep Results:")
print("Thresh | Acc   | Prec  | Rec   | F1    | TP  | TN  | FP  | FN")
print("---------------------------------------------------------------")
y_probs_test = best_model.predict(X_test)
for thresh in np.arange(0.1, 0.7, 0.05):
    y_pred = (y_probs_test > thresh).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    print(f"{thresh:.2f}   | {acc:.3f} | {prec:.3f} | {rec:.3f} | {f1:.3f} | {tp:3d} | {tn:3d} | {fp:3d} | {fn:3d}")


Threshold Sweep Results:
Thresh | Acc   | Prec  | Rec   | F1    | TP  | TN  | FP  | FN
---------------------------------------------------------------


0.10   | 0.913 | 0.861 | 0.998 | 0.925 | 566 | 399 |  91 |   1
0.15   | 0.929 | 0.888 | 0.993 | 0.938 | 563 | 419 |  71 |   4
0.20   | 0.933 | 0.896 | 0.989 | 0.940 | 561 | 425 |  65 |   6
0.25   | 0.935 | 0.903 | 0.984 | 0.942 | 558 | 430 |  60 |   9
0.30   | 0.936 | 0.908 | 0.979 | 0.942 | 555 | 434 |  56 |  12
0.35   | 0.935 | 0.911 | 0.974 | 0.941 | 552 | 436 |  54 |  15
0.40   | 0.938 | 0.918 | 0.970 | 0.943 | 550 | 441 |  49 |  17
0.45   | 0.937 | 0.925 | 0.959 | 0.942 | 544 | 446 |  44 |  23
0.50   | 0.939 | 0.933 | 0.954 | 0.943 | 541 | 451 |  39 |  26
0.55   | 0.937 | 0.937 | 0.945 | 0.941 | 536 | 454 |  36 |  31
0.60   | 0.934 | 0.943 | 0.933 | 0.938 | 529 | 458 |  32 |  38
0.65   | 0.928 | 0.946 | 0.919 | 0.932 | 521 | 460 |  30 |  46


In [11]:
with open(f"./models_dro/{antibiogram['antibiotic'].iloc[0].split(' ')[0]}_xgboost_kmer_{best_kmer_size}_dro.pkl", 'wb') as f:
    	pickle.dump(best_model, f)


In [12]:
###Cefotaxime
antibiogram_file="/data/dorothy/amr_ML/models_DRO/models_hypatia_beforeDeletion/cefotaxime_QCed_metadata.csv"
output="./kmer_rf_xgboost_output.txt"
output_final="./models_dro/kmer_rf_xgboost_output_final.txt"

# -------------------------
# Load the data
# -------------------------
antibiogram = pd.read_csv(antibiogram_file)
antibiogram['phenotype'].value_counts()

print(f"Preparing data for antibiotic: {antibiogram['antibiotic'].iloc[0]}")
print("-" * 40)

with open(output, 'w') as f:
    print(f"Preparing data for antibiotic: {antibiogram['antibiotic'].iloc[0]}", file=f)
    print("-" * 40, file=f)

# Remove antibiograms N phenotype
antibiogram = antibiogram[antibiogram['phenotype'] != 'N']
antibiogram = antibiogram[antibiogram['phenotype'] != 'I']
antibiogram = antibiogram[antibiogram['phenotype'] != 'NS']

antibiogram_unique = antibiogram.drop_duplicates(subset='id')

# Convert 'Element' to binary values
antibiotic_binary = antibiogram_unique['phenotype'].map({'S': 0, 'R': 1})
antibiotic_binary = antibiotic_binary.to_numpy().astype(np.int8)


# Find the indices of the elements in the sublist
indices = [assemblies.index(element) for element in antibiogram_unique['id']]

#New
# Resetting index for antibiogram_gentamycin_unique
antibiogram_unique.reset_index(drop=True, inplace=True)
antibiogram_unique['organism_name'] = antibiogram_unique['organism_name'].replace('E.coli and', 'Escherichia coli')
indices_antibiograms=antibiogram_unique.index.tolist()

#New
# Extract the genus (first word) from the organism_name column
antibiogram_unique['genus'] = antibiogram_unique['organism_name'].str.split().str[0]

# Find how many from each genus are there
genera_counts = antibiogram_unique['genus'].value_counts()

# Keep the indexes of each found genus
genera_indices = antibiogram_unique.groupby('genus').apply(lambda x: x.index.tolist())

# Report total number of rows
total_rows = antibiogram_unique.shape[0]


# Get the antibiotic from the first row
antibiotic_name = antibiogram_unique.loc[0, 'antibiotic']  # Assuming 'antibiotic' is the column name

# List of genera to include in the output
genera_list = ['Escherichia', 'Klebsiella', 'Acinetobacter', 'Staphylococcus', 'Pseudomonas', 'Enterobacter', 'Enterococcus']

# Example genera_counts data (replace with actual values from your dataset)
genera_counts = antibiogram_unique['genus'].value_counts()  # Replace df with your actual DataFrame

# Dictionary to store the results
summary_data = {
    'Total number': [total_rows],  # Total number of rows in your dataset
    'Antibiotic': [antibiotic_name]  # Add the antibiotic name as a row
}

# Populate the dictionary with genus counts, defaulting to 0 if the genus is not present
for genus in genera_list:
    summary_data[genus] = [genera_counts.get(genus, 0)]
    genus_res = antibiogram_unique.loc[antibiogram_unique["genus"]==genus,"phenotype"].value_counts()
    column_wanted=f"{genus}_NumberOfResistantStrains"
    summary_data[column_wanted] = [genus_res.get("R", 0)]
    
# Convert the dictionary to a DataFrame
summary_df = pd.DataFrame(summary_data)

best_kmer_size = 4

X_train, X_test, y_train, y_test, kmer_size, train_indices, test_indices = load_data(best_kmer_size)





import numpy as np
import xgboost as xgb
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

# Split off validation set from training
X_train2, X_val, y_train2, y_val = train_test_split(
    X_train, y_train, test_size=0.2, stratify=y_train, random_state=42
)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from itertools import product

# Define grid
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [4, 6, 8],
    'alpha': [0.2, 0.3],
    'smoothing': [0.9, 0.95]
}

best_score = None
best_model = None
best_params = {}
best_thresh = None

# Grid search over all combinations
for n_est, depth, alpha, smooth in product(param_grid['n_estimators'], param_grid['max_depth'], param_grid['alpha'], param_grid['smoothing']):
    
    # Step 1: Uncertainty weighting
    base_model = xgb.XGBClassifier(n_estimators=100, max_depth=13, random_state=42)
    base_model.fit(X_train2, y_train2)
    y_probs = base_model.predict_proba(X_train2)[:, 1]
    uncertainty = np.abs(0.5 - y_probs)

    cutoff = int(len(y_train2) * alpha)
    worst_indices = np.argsort(uncertainty)[:cutoff]

    sample_weights = np.ones_like(y_train2)
    sample_weights[worst_indices] = 5
    sample_weights[y_train2 == 0] *= 2

    y_smooth = y_train2 * smooth + (1 - smooth) / 2

    # Step 2: DRO model
    model = xgb.XGBRegressor(n_estimators=n_est, max_depth=depth, objective='reg:logistic', random_state=42)
    model.fit(X_train2, y_smooth, sample_weight=sample_weights)

    # Step 3: Threshold sweep with custom F1–FN tradeoff
    lambda_fn = 0.01  # penalty for each FN
    y_probs_test = model.predict(X_test)

    for thresh in np.arange(0.1, 0.7, 0.01):
        y_pred = (y_probs_test > thresh).astype(int)
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        f1 = f1_score(y_test, y_pred)

        # Custom score: prioritize F1, penalize FN
        score = f1 - lambda_fn * fn

        if best_score is None or score > best_score:
            best_score = score
            best_model = model
            best_thresh = thresh
            best_params = {
                'n_estimators': n_est,
                'max_depth': depth,
                'alpha': alpha,
                'smoothing': smooth,
                'threshold': thresh,
                'FP': fp,
                'FN': fn,
                'F1': f1
            }

# Final report
if best_model:
    print(f"\n✅ Best config:")
    for k, v in best_params.items():
        print(f"{k}: {v}")
    y_pred_final = (best_model.predict(X_test) > best_thresh).astype(int)
    print("\nFinal Performance:")
    print(classification_report(y_test, y_pred_final, target_names=["S", "R"]))
else:
    print("❌ No model found (this shouldn't happen).")
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

print("\nThreshold Sweep Results:")
print("Thresh | Acc   | Prec  | Rec   | F1    | TP  | TN  | FP  | FN")
print("---------------------------------------------------------------")
y_probs_test = best_model.predict(X_test)
for thresh in np.arange(0.1, 0.7, 0.05):
    y_pred = (y_probs_test > thresh).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    print(f"{thresh:.2f}   | {acc:.3f} | {prec:.3f} | {rec:.3f} | {f1:.3f} | {tp:3d} | {tn:3d} | {fp:3d} | {fn:3d}")
# Add this code after the threshold sweep results printing
with open(f"./models_dro/{antibiogram['antibiotic'].iloc[0].split(' ')[0]}_threshold_sweep_results.txt", 'w') as f:
    print("\nThreshold Sweep Results:", file=f)
    print("Thresh | Acc   | Prec  | Rec   | F1    | TP  | TN  | FP  | FN", file=f)
    print("---------------------------------------------------------------", file=f)
    y_probs_test = best_model.predict(X_test)
    for thresh in np.arange(0.1, 0.7, 0.05):
        y_pred = (y_probs_test > thresh).astype(int)
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        
        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred, zero_division=0)
        rec = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        
        print(f"{thresh:.2f}   | {acc:.3f} | {prec:.3f} | {rec:.3f} | {f1:.3f} | {tp:3d} | {tn:3d} | {fp:3d} | {fn:3d}", file=f)
        
        
with open(f"./models_dro/{antibiogram['antibiotic'].iloc[0].split(' ')[0]}_xgboost_kmer_{best_kmer_size}_dro.pkl", 'wb') as f:
    	pickle.dump(best_model, f)


Preparing data for antibiotic: cefotaxime
----------------------------------------


/tmp/ipykernel_3170338/4284624723.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  antibiogram_unique['organism_name'] = antibiogram_unique['organism_name'].replace('E.coli and', 'Escherichia coli')
/tmp/ipykernel_3170338/4284624723.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  antibiogram_unique['genus'] = antibiogram_unique['organism_name'].str.split().str[0]
/tmp/ipykernel_3170338/4284624723.py:48: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is de

Loading data for kmer size 4

✅ Best config:
n_estimators: 100
max_depth: 4
alpha: 0.2
smoothing: 0.95
threshold: 0.1
FP: 18
FN: 2
F1: 0.9765258215962441

Final Performance:
              precision    recall  f1-score   support

           S       0.99      0.90      0.94       187
           R       0.96      1.00      0.98       418

    accuracy                           0.97       605
   macro avg       0.97      0.95      0.96       605
weighted avg       0.97      0.97      0.97       605


Threshold Sweep Results:
Thresh | Acc   | Prec  | Rec   | F1    | TP  | TN  | FP  | FN
---------------------------------------------------------------
0.10   | 0.967 | 0.959 | 0.995 | 0.977 | 416 | 169 |  18 |   2
0.15   | 0.970 | 0.972 | 0.986 | 0.979 | 412 | 175 |  12 |   6
0.20   | 0.972 | 0.976 | 0.983 | 0.980 | 411 | 177 |  10 |   7
0.25   | 0.972 | 0.976 | 0.983 | 0.980 | 411 | 177 |  10 |   7
0.30   | 0.972 | 0.979 | 0.981 | 0.980 | 410 | 178 |   9 |   8
0.35   | 0.972 | 0.981 | 0.978 |

In [8]:
###Cefotaxime
antibiogram_file="/data/dorothy/amr_ML/models_DRO/models_hypatia_beforeDeletion/gentamicin_QCed_metadata.csv"
output="./kmer_rf_xgboost_output.txt"
output_final="./models_dro/kmer_rf_xgboost_output_final.txt"

# -------------------------
# Load the data
# -------------------------
antibiogram = pd.read_csv(antibiogram_file)
antibiogram['phenotype'].value_counts()

print(f"Preparing data for antibiotic: {antibiogram['antibiotic'].iloc[0]}")
print("-" * 40)

with open(output, 'w') as f:
    print(f"Preparing data for antibiotic: {antibiogram['antibiotic'].iloc[0]}", file=f)
    print("-" * 40, file=f)

# Remove antibiograms N phenotype
antibiogram = antibiogram[antibiogram['phenotype'] != 'N']
antibiogram = antibiogram[antibiogram['phenotype'] != 'I']
antibiogram = antibiogram[antibiogram['phenotype'] != 'NS']

antibiogram_unique = antibiogram.drop_duplicates(subset='id')

# Convert 'Element' to binary values
antibiotic_binary = antibiogram_unique['phenotype'].map({'S': 0, 'R': 1})
antibiotic_binary = antibiotic_binary.to_numpy().astype(np.int8)


# Find the indices of the elements in the sublist
indices = [assemblies.index(element) for element in antibiogram_unique['id']]

#New
# Resetting index for antibiogram_gentamycin_unique
antibiogram_unique.reset_index(drop=True, inplace=True)
antibiogram_unique['organism_name'] = antibiogram_unique['organism_name'].replace('E.coli and', 'Escherichia coli')
indices_antibiograms=antibiogram_unique.index.tolist()

#New
# Extract the genus (first word) from the organism_name column
antibiogram_unique['genus'] = antibiogram_unique['organism_name'].str.split().str[0]

# Find how many from each genus are there
genera_counts = antibiogram_unique['genus'].value_counts()

# Keep the indexes of each found genus
genera_indices = antibiogram_unique.groupby('genus').apply(lambda x: x.index.tolist())

# Report total number of rows
total_rows = antibiogram_unique.shape[0]


# Get the antibiotic from the first row
antibiotic_name = antibiogram_unique.loc[0, 'antibiotic']  # Assuming 'antibiotic' is the column name

# List of genera to include in the output
genera_list = ['Escherichia', 'Klebsiella', 'Acinetobacter', 'Staphylococcus', 'Pseudomonas', 'Enterobacter', 'Enterococcus']

# Example genera_counts data (replace with actual values from your dataset)
genera_counts = antibiogram_unique['genus'].value_counts()  # Replace df with your actual DataFrame

# Dictionary to store the results
summary_data = {
    'Total number': [total_rows],  # Total number of rows in your dataset
    'Antibiotic': [antibiotic_name]  # Add the antibiotic name as a row
}

# Populate the dictionary with genus counts, defaulting to 0 if the genus is not present
for genus in genera_list:
    summary_data[genus] = [genera_counts.get(genus, 0)]
    genus_res = antibiogram_unique.loc[antibiogram_unique["genus"]==genus,"phenotype"].value_counts()
    column_wanted=f"{genus}_NumberOfResistantStrains"
    summary_data[column_wanted] = [genus_res.get("R", 0)]
    
# Convert the dictionary to a DataFrame
summary_df = pd.DataFrame(summary_data)

best_kmer_size = 5

X_train, X_test, y_train, y_test, kmer_size, train_indices, test_indices = load_data(best_kmer_size)





import numpy as np
import xgboost as xgb
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

# Split off validation set from training
X_train2, X_val, y_train2, y_val = train_test_split(
    X_train, y_train, test_size=0.2, stratify=y_train, random_state=42
)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from itertools import product

# Define grid
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [4, 6, 8],
    'alpha': [0.2, 0.3],
    'smoothing': [0.9, 0.95]
}

best_score = None
best_model = None
best_params = {}
best_thresh = None

# Grid search over all combinations
for n_est, depth, alpha, smooth in product(param_grid['n_estimators'], param_grid['max_depth'], param_grid['alpha'], param_grid['smoothing']):
    
    # Step 1: Uncertainty weighting
    base_model = xgb.XGBClassifier(n_estimators=100, max_depth=13, random_state=42)
    base_model.fit(X_train2, y_train2)
    y_probs = base_model.predict_proba(X_train2)[:, 1]
    uncertainty = np.abs(0.5 - y_probs)

    cutoff = int(len(y_train2) * alpha)
    worst_indices = np.argsort(uncertainty)[:cutoff]

    sample_weights = np.ones_like(y_train2)
    sample_weights[worst_indices] = 5
    sample_weights[y_train2 == 0] *= 2

    y_smooth = y_train2 * smooth + (1 - smooth) / 2

    # Step 2: DRO model
    model = xgb.XGBRegressor(n_estimators=n_est, max_depth=depth, objective='reg:logistic', random_state=42)
    model.fit(X_train2, y_smooth, sample_weight=sample_weights)

    # Step 3: Threshold sweep with custom F1–FN tradeoff
    lambda_fn = 0.01  # penalty for each FN
    y_probs_test = model.predict(X_test)

    for thresh in np.arange(0.1, 0.7, 0.01):
        y_pred = (y_probs_test > thresh).astype(int)
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        f1 = f1_score(y_test, y_pred)

        # Custom score: prioritize F1, penalize FN
        score = f1 - lambda_fn * fn

        if best_score is None or score > best_score:
            best_score = score
            best_model = model
            best_thresh = thresh
            best_params = {
                'n_estimators': n_est,
                'max_depth': depth,
                'alpha': alpha,
                'smoothing': smooth,
                'threshold': thresh,
                'FP': fp,
                'FN': fn,
                'F1': f1
            }

# Final report
if best_model:
    print(f"\n✅ Best config:")
    for k, v in best_params.items():
        print(f"{k}: {v}")
    y_pred_final = (best_model.predict(X_test) > best_thresh).astype(int)
    print("\nFinal Performance:")
    print(classification_report(y_test, y_pred_final, target_names=["S", "R"]))
else:
    print("❌ No model found (this shouldn't happen).")
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

print("\nThreshold Sweep Results:")
print("Thresh | Acc   | Prec  | Rec   | F1    | TP  | TN  | FP  | FN")
print("---------------------------------------------------------------")
y_probs_test = best_model.predict(X_test)
for thresh in np.arange(0.1, 0.7, 0.05):
    y_pred = (y_probs_test > thresh).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    print(f"{thresh:.2f}   | {acc:.3f} | {prec:.3f} | {rec:.3f} | {f1:.3f} | {tp:3d} | {tn:3d} | {fp:3d} | {fn:3d}")
# Add this code after the threshold sweep results printing
with open(f"./models_dro/{antibiogram['antibiotic'].iloc[0].split(' ')[0]}_threshold_sweep_results.txt", 'w') as f:
    print("\nThreshold Sweep Results:", file=f)
    print("Thresh | Acc   | Prec  | Rec   | F1    | TP  | TN  | FP  | FN", file=f)
    print("---------------------------------------------------------------", file=f)
    y_probs_test = best_model.predict(X_test)
    for thresh in np.arange(0.1, 0.7, 0.05):
        y_pred = (y_probs_test > thresh).astype(int)
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        
        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred, zero_division=0)
        rec = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        
        print(f"{thresh:.2f}   | {acc:.3f} | {prec:.3f} | {rec:.3f} | {f1:.3f} | {tp:3d} | {tn:3d} | {fp:3d} | {fn:3d}", file=f)
        
        
with open(f"./models_dro/{antibiogram['antibiotic'].iloc[0].split(' ')[0]}_xgboost_kmer_{best_kmer_size}_dro.pkl", 'wb') as f:
    	pickle.dump(best_model, f)


Preparing data for antibiotic: gentamicin
----------------------------------------


/tmp/ipykernel_3183208/2065259621.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  antibiogram_unique['organism_name'] = antibiogram_unique['organism_name'].replace('E.coli and', 'Escherichia coli')
/tmp/ipykernel_3183208/2065259621.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  antibiogram_unique['genus'] = antibiogram_unique['organism_name'].str.split().str[0]
/tmp/ipykernel_3183208/2065259621.py:48: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is de

Loading data for kmer size 5


KeyboardInterrupt: 

In [1]:
###Cefotaxime
antibiogram_file="/data/dorothy/amr_ML/models_DRO/models_hypatia_beforeDeletion/ampicillin_QCed_metadata.csv"
output="./kmer_rf_xgboost_output.txt"
output_final="./models_dro/kmer_rf_xgboost_output_final.txt"

# -------------------------
# Load the data
# -------------------------
antibiogram = pd.read_csv(antibiogram_file)
antibiogram['phenotype'].value_counts()

print(f"Preparing data for antibiotic: {antibiogram['antibiotic'].iloc[0]}")
print("-" * 40)

with open(output, 'w') as f:
    print(f"Preparing data for antibiotic: {antibiogram['antibiotic'].iloc[0]}", file=f)
    print("-" * 40, file=f)

# Remove antibiograms N phenotype
antibiogram = antibiogram[antibiogram['phenotype'] != 'N']
antibiogram = antibiogram[antibiogram['phenotype'] != 'I']
antibiogram = antibiogram[antibiogram['phenotype'] != 'NS']

antibiogram_unique = antibiogram.drop_duplicates(subset='id')

# Convert 'Element' to binary values
antibiotic_binary = antibiogram_unique['phenotype'].map({'S': 0, 'R': 1})
antibiotic_binary = antibiotic_binary.to_numpy().astype(np.int8)


# Find the indices of the elements in the sublist
indices = [assemblies.index(element) for element in antibiogram_unique['id']]

#New
# Resetting index for antibiogram_gentamycin_unique
antibiogram_unique.reset_index(drop=True, inplace=True)
antibiogram_unique['organism_name'] = antibiogram_unique['organism_name'].replace('E.coli and', 'Escherichia coli')
indices_antibiograms=antibiogram_unique.index.tolist()

#New
# Extract the genus (first word) from the organism_name column
antibiogram_unique['genus'] = antibiogram_unique['organism_name'].str.split().str[0]

# Find how many from each genus are there
genera_counts = antibiogram_unique['genus'].value_counts()

# Keep the indexes of each found genus
genera_indices = antibiogram_unique.groupby('genus').apply(lambda x: x.index.tolist())

# Report total number of rows
total_rows = antibiogram_unique.shape[0]


# Get the antibiotic from the first row
antibiotic_name = antibiogram_unique.loc[0, 'antibiotic']  # Assuming 'antibiotic' is the column name

# List of genera to include in the output
genera_list = ['Escherichia', 'Klebsiella', 'Acinetobacter', 'Staphylococcus', 'Pseudomonas', 'Enterobacter', 'Enterococcus']

# Example genera_counts data (replace with actual values from your dataset)
genera_counts = antibiogram_unique['genus'].value_counts()  # Replace df with your actual DataFrame

# Dictionary to store the results
summary_data = {
    'Total number': [total_rows],  # Total number of rows in your dataset
    'Antibiotic': [antibiotic_name]  # Add the antibiotic name as a row
}

# Populate the dictionary with genus counts, defaulting to 0 if the genus is not present
for genus in genera_list:
    summary_data[genus] = [genera_counts.get(genus, 0)]
    genus_res = antibiogram_unique.loc[antibiogram_unique["genus"]==genus,"phenotype"].value_counts()
    column_wanted=f"{genus}_NumberOfResistantStrains"
    summary_data[column_wanted] = [genus_res.get("R", 0)]
    
# Convert the dictionary to a DataFrame
summary_df = pd.DataFrame(summary_data)

best_kmer_size = 4

X_train, X_test, y_train, y_test, kmer_size, train_indices, test_indices = load_data(best_kmer_size)





import numpy as np
import xgboost as xgb
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

# Split off validation set from training
X_train2, X_val, y_train2, y_val = train_test_split(
    X_train, y_train, test_size=0.2, stratify=y_train, random_state=42
)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from itertools import product

# Define grid
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [4, 6, 8],
    'alpha': [0.2, 0.3],
    'smoothing': [0.9, 0.95]
}

best_score = None
best_model = None
best_params = {}
best_thresh = None

# Grid search over all combinations
for n_est, depth, alpha, smooth in product(param_grid['n_estimators'], param_grid['max_depth'], param_grid['alpha'], param_grid['smoothing']):
    
    # Step 1: Uncertainty weighting
    base_model = xgb.XGBClassifier(n_estimators=100, max_depth=13, random_state=42)
    base_model.fit(X_train2, y_train2)
    y_probs = base_model.predict_proba(X_train2)[:, 1]
    uncertainty = np.abs(0.5 - y_probs)

    cutoff = int(len(y_train2) * alpha)
    worst_indices = np.argsort(uncertainty)[:cutoff]

    sample_weights = np.ones_like(y_train2)
    sample_weights[worst_indices] = 5
    sample_weights[y_train2 == 0] *= 2

    y_smooth = y_train2 * smooth + (1 - smooth) / 2

    # Step 2: DRO model
    model = xgb.XGBRegressor(n_estimators=n_est, max_depth=depth, objective='reg:logistic', random_state=42)
    model.fit(X_train2, y_smooth, sample_weight=sample_weights)

    # Step 3: Threshold sweep with custom F1–FN tradeoff
    lambda_fn = 0.01  # penalty for each FN
    y_probs_test = model.predict(X_test)

    for thresh in np.arange(0.1, 0.7, 0.01):
        y_pred = (y_probs_test > thresh).astype(int)
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        f1 = f1_score(y_test, y_pred)

        # Custom score: prioritize F1, penalize FN
        score = f1 - lambda_fn * fn

        if best_score is None or score > best_score:
            best_score = score
            best_model = model
            best_thresh = thresh
            best_params = {
                'n_estimators': n_est,
                'max_depth': depth,
                'alpha': alpha,
                'smoothing': smooth,
                'threshold': thresh,
                'FP': fp,
                'FN': fn,
                'F1': f1
            }

# Final report
if best_model:
    print(f"\n✅ Best config:")
    for k, v in best_params.items():
        print(f"{k}: {v}")
    y_pred_final = (best_model.predict(X_test) > best_thresh).astype(int)
    print("\nFinal Performance:")
    print(classification_report(y_test, y_pred_final, target_names=["S", "R"]))
else:
    print("❌ No model found (this shouldn't happen).")
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

print("\nThreshold Sweep Results:")
print("Thresh | Acc   | Prec  | Rec   | F1    | TP  | TN  | FP  | FN")
print("---------------------------------------------------------------")
y_probs_test = best_model.predict(X_test)
for thresh in np.arange(0.1, 0.7, 0.05):
    y_pred = (y_probs_test > thresh).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    print(f"{thresh:.2f}   | {acc:.3f} | {prec:.3f} | {rec:.3f} | {f1:.3f} | {tp:3d} | {tn:3d} | {fp:3d} | {fn:3d}")
# Add this code after the threshold sweep results printing
with open(f"./models_dro/{antibiogram['antibiotic'].iloc[0].split(' ')[0]}_threshold_sweep_results.txt", 'w') as f:
    print("\nThreshold Sweep Results:", file=f)
    print("Thresh | Acc   | Prec  | Rec   | F1    | TP  | TN  | FP  | FN", file=f)
    print("---------------------------------------------------------------", file=f)
    y_probs_test = best_model.predict(X_test)
    for thresh in np.arange(0.1, 0.7, 0.05):
        y_pred = (y_probs_test > thresh).astype(int)
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        
        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred, zero_division=0)
        rec = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        
        print(f"{thresh:.2f}   | {acc:.3f} | {prec:.3f} | {rec:.3f} | {f1:.3f} | {tp:3d} | {tn:3d} | {fp:3d} | {fn:3d}", file=f)
        
        
with open(f"./models_dro/{antibiogram['antibiotic'].iloc[0].split(' ')[0]}_xgboost_kmer_{best_kmer_size}_dro.pkl", 'wb') as f:
    	pickle.dump(best_model, f)


NameError: name 'pd' is not defined